# Libraries

In [1]:
import tensorflow as tf
import gym
import numpy as np
from stable_baselines import DQN #type: ignore
from copy import deepcopy
import math
from gym.spaces import Discrete, Dict, Box
from gym import spaces
from random import seed
import random 
from gym import Env
from datetime import datetime
import sys
import time
import pickle
import stable_baselines
import sklearn
from sklearn import tree , svm 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB , CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix #type: ignore
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from itertools import product
from sklearn.utils import resample
from sklearn.model_selection import KFold , RepeatedKFold
from sklearn.metrics import f1_score
from sklearn import impute
import statistics
from scipy import stats
from copy import deepcopy
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from math import ceil
import copy
import sys
from sklearn.metrics import jaccard_score
import time
import multiprocessing
from pymoo.algorithms.nsga2 import calc_crowding_distance
sys.path.append('lib/')
import subprocess
import logging
from sklearn.utils import shuffle
import csv
from csv import reader
import os
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score


class StoreWrapper(gym.Wrapper):
  ''''
  :param env: (gym.Env) Gym environment that will be wrapped
  :param max_steps: (int) Max number of steps per episode
  '''
  def __init__(self, env):
    # Call the parent constructor, so we can access self.env later
    super(StoreWrapper, self).__init__(env)
    self.max_steps = 200
    # Counter of steps per episode
    self.current_step = 0
    self.mem = []
    self.TotalReward = 0.0 
    self.env = env
    self.first_state = 0
    self.first_obs = 0
    self.prev_obs = 0 
    self.states_list = []
    self.info = {}
  
  def reset(self):
    """
    Reset the environment 
    """
    # Reset the counter
    self.current_step = 0
    obs =self.env.reset()
    self.TotalReward = 0.0
    self.first_obs = obs
    return obs

  def step(self, action):
    """
    In this function we store the initial state as well as the memory of the agent
    :param action: ([float] or int) Action taken by the agent
    :return: (np.ndarray, float, bool, dict) observation, reward, is the episode over?, additional informations
    """
    if self.current_step == 0: #store initial state
      self.prev_obs = self.first_obs
      self.first_state = deepcopy(self.env)
      self.states_list.append(self.first_state)
    self.current_step += 1
    obs, reward, done, info = self.env.step(action)
    self.TotalReward += reward
    self.mem.append(tuple((self.prev_obs,action)))
    self.prev_obs = obs
    if self.current_step >= self.max_steps:
      done = True
      # Update the info dict to signal that the limit was exceeded
    if done:
      self.mem.append(tuple(('done',self.TotalReward)))
    self.info['mem'] = self.mem
    self.info['state'] = self.states_list
    # self.mem.append(tuple(obs,action))
    return obs, reward, done, info

  def set_state(self, state):
    """
    :param state: initial state of the episode
    :return: environment is updated and observations is returned
    """
    self.env = deepcopy(state)
    obs = np.array(list(self.env.unwrapped.state))
    self.current_step = 0
    self.TotalReward = 0.0
    self.first_obs = obs
    return obs

def abstract_state(model,state1,d):
  if type(state1) == str:
    if state1 == 'done':
      return 'end'
  q_value1 = model.step_model.step([state1])
  return( ceil(q_value1[1][0][0]/d), ceil(q_value1[1][0][1]/d))

def abstract_state_general(model,state1,d):
  if type(state1) == str:
    if state1 == 'done':
      return 'end'
  q_values = model.step_model.step([state1])
  return tuple([ceil(q_value/d) for q_value in q_values[1][0]])

def Abstract_classes(ep,abstraction_d,model):
  d=abstraction_d
  abs_states1=[]
  for episode in ep:
    for state,action in episode:
      abs_st = abstract_state(model,state,d)
      if abs_st == 'end':
        continue
      abs_states1.append(abs_st)
  unique1=list(set(abs_states1))
  uni1 = np.array(unique1)
  a=len(abs_states1)
  b=len(set(abs_states1))
  print("abstract states:",b)
  print("Concrete states",a)
  print("ratio",b/a)
  return unique1,uni1

def is_functional_fault(episode,epsilon):
  for state, _ in episode:
    if type(state) == str:
      if state == 'done':
        return False
    if abs(state[0]) >= (2.4-epsilon):
      return True
  # no functional fault is obsereved
  return False

def is_reward_fault(episode,threshold):
  last_state = episode[-1]
  assert last_state[0] == 'done'
  if last_state[1] < threshold:
    return True
  else:
    return False

def ML_first_representation_func_based(Abs_d,functional_func,reward_func,model,input_episodes,unique1,epsilon,threshold):
  """
  TO-DO : fix epsilon and threshold
  """
  d = Abs_d
  data1_x_b=[]
  data1_y_b= [] 
  data1_y_f_b = []
  for i, episode in enumerate(input_episodes):
    record = np.zeros(len(unique1))
    temp_flag = False
    for state, action in episode:
      ab = abstract_state(model,state,d)
      if ab == 'end':
        assert not temp_flag, f'Episode data problem, two terminations in one episode. Episode number{i}'
        temp_flag = True
        # print(action)
        # print(functional_func(episode))
        if functional_func(episode,epsilon):
          data1_y_f_b.append(1)
        else:
          data1_y_f_b.append(0)
        if reward_func(episode,threshold):
          data1_y_b.append(1)
        else:
          data1_y_b.append(0)
        # print("end\n\n\n")
        # print(len(data1_y_b),"len(input_episodes)",len(input_episodes))
        continue
        # print(state[0])
      ind = unique1.index(ab)
      record[ind] = 1
      # print(state, action)
      assert len(data1_y_b)<len(input_episodes), "assert"
      # if you want the frequency go with the next line 
      # record[ind] += 1
    data1_x_b.append(record)

  return data1_x_b, data1_y_b, data1_y_f_b


def ML_first_representation(Abs_d,epsilon_functional_fault_boarder,model,ep,unique1):
  d = Abs_d
  # epsilon = 0.05
  epsilon = epsilon_functional_fault_boarder
  data1_x_b=[]
  data1_y_b= [] 
  data1_y_f_b = []
  functional_fault = False
  reward_fault_threshold = 70
  for episode in ep:
    record = np.zeros(len(unique1))
    for state, action in episode:
      ab = abstract_state(model,state,d)
      if ab == 'end':
        # print(action)
        if functional_fault:
          data1_y_f_b.append(1)
        else:
          data1_y_f_b.append(0)
        if action >= reward_fault_threshold:
          data1_y_b.append(0)
        else:
          data1_y_b.append(1)
        functional_fault=False
        continue
      if abs(state[0]) >= (2.4-epsilon) :
        functional_fault = True
      ind = unique1.index(ab)
      # if len(w[0])>1:
        # print('error len is greater than 1')
      record[ind] = 1
      # if you want the frequency go with the next line 
      # record[ind] += 1
    data1_x_b.append(record)

  return data1_x_b, data1_y_b, data1_y_f_b

#report function to check the performance metrics of the model
def report(model2,x_train, y_train,x_test, y_test):
  print("********************** reporting the result of the model **************************")
  print('The score for train data is {0}'.format(model2.score(x_train,y_train)))
  print('The score for test data is {0}'.format(model2.score(x_test,y_test)))


  predictions_train = model2.predict(x_train)
  predictions_test = model2.predict(x_test)

  print("\n\n--------------------------------------recall---------------------------------")

  print('the test recall for the class yes is {0}'.format(metrics.recall_score(y_test,predictions_test, pos_label=1)))
  print('the test recall for the class no is {0}'.format(metrics.recall_score(y_test,predictions_test, pos_label=0)))

  print('the training recall for the class yes is {0}'.format(metrics.recall_score(y_train,predictions_train, pos_label=1)))
  print('the training recall for the class no is {0}'.format(metrics.recall_score(y_train,predictions_train, pos_label=0)))


  print("\n\n--------------------------------------precision------------------------------")


  print('the test precision for the class yes is {0}'.format(metrics.precision_score(y_test,predictions_test, pos_label=1)))
  print('the test precision for the class no is {0}'.format(metrics.precision_score(y_test,predictions_test, pos_label=0)))

  print('the training precision for the class yes is {0}'.format(metrics.precision_score(y_train,predictions_train, pos_label=1)))
  print('the training precision for the class no is {0}'.format(metrics.precision_score(y_train,predictions_train, pos_label=0)))

  print("\n\n")
  print(classification_report(y_test, predictions_test, target_names=['NO ','yes']))

  tn, fp, fn, tp = confusion_matrix(y_test, predictions_test).ravel()
  specificity = tn / (tn+fp)
  print("\n\nspecifity :",specificity)
  print("\n\n--------------------------------------confusion----------------------------")
  CM = metrics.confusion_matrix(y_test, predictions_test)
  print("The confusion Matrix:")
  print(CM)
  print('the accuracy score in {0}\n\n'.format(accuracy_score(y_test, predictions_test)))
  print("********************** plotting the confusion matrix & ROC curve **************************")
  plot_confusion_matrix(model2, x_test, y_test) #type: ignore
  metrics.plot_roc_curve(model2, x_test, y_test)  #type: ignore
  plt.show()



def random_test_2(model, env, Num):
  obs=env.reset()
  counter = 1
  episode_reward = 0.0
  end =-1
  lastpoint = -1
  for i in range(Num):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    episode_reward += reward
    if done:
      counter += 1
      end = i
      episode_reward = 0.0
      obs = env.reset()
  iter = deepcopy(counter)
  u=1
  while iter>1:
    if env.info['mem'][-u][0]=='done': 
      lastpoint = -u
      iter -= 1
    u+=1
  assert lastpoint != -1
  assert end != -1
  fin =Num - end
  start = -Num -counter
  randomtest = env.info['mem'][lastpoint:-fin] 
  ran_state = env.info['state'][(-counter+1):-1] 
  return randomtest , ran_state

def fix_testing(testing_episodes,testing_states,Env2):
  # TO DO: fix data typr for if condition change to nparray
  buffer =[] 
  episodes_set = []
  j=0
  for i in range(len(testing_episodes)):
    if testing_episodes[i][0] == 'done':
      if i == 0:
        continue
      buffer.append(testing_episodes[i])
      episodes_set.append(buffer)
      buffer=[]
    else:
      buffer.append(testing_episodes[i])
  if not (np.array(episodes_set[0][0][0])== np.array(Env2.set_state(testing_states[0]),dtype="float32")).all():
    del testing_states[0]
  if not (np.array(episodes_set[0][0][0])==np.array(Env2.set_state(testing_states[0]),dtype="float32")).all():
    assert False, 'problem in starting states'
  if len(episodes_set)!=len(testing_states):
    del testing_states[-1]
  if len(episodes_set)!=len(testing_states):
    assert False, 'problem in data prepration'
  return episodes_set , testing_states





c:\Users\Student\Desktop\vs_git\.virtualenvs\venv\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



c:\Users\Student\Desktop\vs_git\.virtualenvs\venv\lib\site-packages\stable_baselines\__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [3]:
#Address of the trained RL model 

Dataset_path = "Path to the dataset"

Drive_model  =f"{Dataset_path}/Trained_agent/dqn-168.pkl"


env2 = gym.make('CartPole-v1')
env2 = StoreWrapper(env2)
model = DQN('MlpPolicy',env=env2, verbose=1)
model = model.load(Drive_model)
#########################################################  Read DATA and Load Model #############


Loading a model without an environment, this model cannot be trained until it has a valid environment.


# Plot functions

In [4]:
import scipy

def translator(episode,model, d, unique5):
  """
  this function takes the concrete episodes and returns the encoded episodes 
  based on the presence and absence of the individuals  
  :param 'episode': input episode
  :param 'model': RL model
  :param 'd': abstraction level = 1
  :param 'unique5': abstract classes 
  :return: encoded episodse based on the presence and absence

  """
  d=d
  record = np.zeros(len(unique5))
  for state, action in episode:
    ab = abstract_state(model,state,d)
    if ab == 'end':
      continue
    if ab in unique5:
      ind = unique5.index(ab)
      record[ind] = 1
  return [record]

def episode_player(episodes,d, abs_classes, model, monitor) -> list:
  ''' This function replays the episodes and returns the risk of each step in each episode
  :param 'episodes': input episodes
  :param 'd': abstraction level 
  :param 'abs_classes': abstract classes
  :param 'model': RL model
  :param 'monitor': ML model
  :return: risk of each step in each episode
  
  '''
  episodes_risk=[]
  for episode in episodes:
    risk_array=[]
    for step in range(len(episode)-1):
      monitoring_data = translator(episode[:step],model,d,abs_classes)
      Risk = monitor.predict_proba(monitoring_data)
      risk_array.append(Risk[0][1])
    episodes_risk.append(risk_array)
  return episodes_risk

def single_episode_player(episode,d, abs_classes, model, monitor) -> list:
  ''' This function replays one episodes and returns the risk of each step in episode
  :param 'episode': input episode
  :param 'd': abstraction level
  :param 'abs_classes': abstract classes
  :param 'model': RL model
  :param 'monitor': ML model
  :return: risk of each step in episode
  '''
  risk_array=[]
  for step in range(len(episode)-1):
    monitoring_data = translator(episode[:step],model,d,abs_classes)
    Risk = monitor.predict_proba(monitoring_data)
    risk_array.append(Risk[0][1])
  return risk_array


def position_extractor(episode):
    position =[]
    for i in range(len(episode)-1):
        position.append(episode[i][0][0])
    return position
    
def velocity_extractor(episode):
    velocity =[]
    for i in range(len(episode)-1):
        velocity.append(episode[i][0][1])
    return velocity
def angle_extractor(episode):
    angle =[]
    for i in range(len(episode)-1):
        angle.append(episode[i][0][2])
    return angle
def angular_v_extractor(episode):
    angular_v =[]
    for i in range(len(episode)-1):
        angular_v.append(episode[i][0][3])
    return angular_v

from tqdm import tqdm

def Plot_all(data,abstraction_level,unique1,RL_model,Monitor,save=False,show=True,data_chunk=0,path='Plots_CartPole'):
    '''plot risk and position snd velocity in one figure with 3 subplots
    '''
    ## TODO: write a function to return the setteings of the environment and the model

    fig, axs = plt.subplots(5,figsize=(20, 18))
    for i in range(len(data)):
        axs[0].plot([i for i in range(len(data[i])-1)], single_episode_player(data[i],abstraction_level,unique1,RL_model,Monitor), label = f"Episode {i}")
        axs[1].plot([i for i in range(len(data[i])-1)], position_extractor(data[i]), label = f"Episode {i}")
        axs[2].plot([i for i in range(len(data[i])-1)], velocity_extractor(data[i]), label = f"Episode {i}")
        axs[3].plot([i for i in range(len(data[i])-1)], angle_extractor(data[i]), label = f"Episode {i}")
        axs[4].plot([i for i in range(len(data[i])-1)], angular_v_extractor(data[i]) ,label = f"Episode {i}")
    axs[0].legend()
    #plot title and labels
    axs[0].set_title('Risk')
    axs[1].legend()
    axs[1].set_title('Position')
    axs[2].set_title('Velocity')
    axs[3].set_title('Angle')
    axs[4].set_title('Angular Velocity')
    axs[2].legend()
    axs[3].legend()
    axs[4].legend()
    #set range of y axis
    axs[0].set_ylim(-0.1, 1.1)
    
    current_time = datetime.now()
    ID = current_time.strftime("%Y%m%d%H%M%S")
    if save:
        fig.savefig(f'{path}/RPVAA2_C{data_chunk}_{ID}.png')
    #close the plot
    plt.close()

def translate_episode_steps(episode,HD_model,translator,d,abs_classes):
    translated_episode = []
    for i in range(len(episode)-1):
        translated_episode.append(translator(episode[:i],HD_model,d,abs_classes))
    return translated_episode

def translate_multiple_episodes_steps(episodes,HD_model,translator,d,abs_classes):
    translated_episodes = []
    for episode in episodes:
        translated_buffer = []
        for i in range(len(episode)-1):
            translated_buffer.append(translator(episode[:i],HD_model,d,abs_classes))
        translated_episodes.append(translated_buffer)
    return translated_episodes




def Forest_CI_multiple(translated_episodes,HD_model,chunk,abs_d,path = '/home'):
    '''
    size of translated_episodes is limited by the number of colors available for one plot
    '''
    plt.figure(figsize=(20, 12))
    results_Arr=[]
    r_arr=[]
    E=0
    colors = ['red','blue','green','yellow','black','purple','orange','pink','brown','grey','cyan','magenta','lime','olive','teal','navy','maroon','violet','turquoise','salmon','gold','coral','indigo','crimson','azure','beige','chocolate','lavender','plum','orchid','tan','khaki','wheat','silver','sienna','peru','peachpuff','papayawhip','mistyrose','moccasin','lemonchiffon','lawngreen','lightgreen','limegreen']
    for translated_episode , plt_color in zip(translated_episodes,colors[:len(translated_episodes)]):
        E+=1
        num_time_steps = len(translated_episode) # Number of time steps
        num_trees = HD_model.n_estimators
        predictions = np.zeros((num_time_steps, num_trees))
        for i, tree in enumerate(HD_model.estimators_):
            for j in range(len(translated_episode)):
                Risk = tree.predict_proba(translated_episode[j])[0][1]
                predictions[j, i] = Risk
        # Calculate the mean prediction for each time step
        mean_predictions = np.mean(predictions, axis=1)

        # Calculate the standard deviation for each time step
        std_predictions = np.std(predictions, axis=1)

        # Calculate the lower and upper bounds for the confidence intervals
        confidence_level = 0.95 # Change as needed
        z_score = scipy.stats.norm.ppf((1 + confidence_level) / 2)
        lower_bounds = mean_predictions - z_score * std_predictions / np.sqrt(num_trees)
        upper_bounds = mean_predictions + z_score * std_predictions / np.sqrt(num_trees)
        difference = upper_bounds - lower_bounds
        # Store the results in a dataframe
        results = pd.DataFrame({
            'Mean prediction': mean_predictions,
            'Lower bound': lower_bounds,
            'Upper bound': upper_bounds
        })
        # Save the results to a file
        results_Arr.append(results)
        r_arr.append([mean_predictions, lower_bounds, upper_bounds,difference])
        plt.fill_between(range(num_time_steps), lower_bounds, upper_bounds, color=plt_color, alpha=0.2)
        plt.plot(mean_predictions, color=plt_color, label=f'Episode {E}')
        # set the y axis limits
        plt.ylim(-0.1, 1.1)
        # Add labels and title to the plot
        plt.xlabel('Time step')
        plt.ylabel('Prediction')
        plt.title('Confidence Intervals of Random Forest Predictions')
    # results_arr.append(r_arr)
    # Save the plot as a file
    current_time = datetime.now()
    ID = current_time.strftime("%Y%m%d%H%M%S")
    plt.legend()
    plt.savefig(f'{path}/confidence_intervals_{chunk}_{ID}.png', bbox_inches='tight')
    plt.close()
    pickle_path = f'CI/Abs_{abs_d}'
    if not os.path.exists(pickle_path):
        os.makedirs(pickle_path)
    with open(f'{pickle_path}/results_arr_{chunk}.pkl', 'wb') as f:
        pickle.dump(r_arr, f)




    


# Run Confidence Intervals Plot

In [ ]:
d_set=[5,4,3,2,1.5,1,0.8,0.5,0.1,0.05,0.01,0.005]


Drive_model  =f"{Dataset_path}/Trained_agent/dqn-168.pkl"


env2 = gym.make('CartPole-v1')
env2 = StoreWrapper(env2)
model = DQN('MlpPolicy',env=env2, verbose=1)
model = model.load(Drive_model)



for d in d_set:
    print(f'\n\n\n#########################################\nAbstraction level: {d}')
    # unique1 = CartPole_config.unique1
    #read unique1 from pickle
    Read_from_data = True
    if Read_from_data:
        # assert False
        with open(f'{Dataset_path}/ABS/Abstraction_data_final_{d}.pickle', 'rb') as file2:
            unique1 = pickle.load(file2)
        uni1=np.array(unique1)
    else:
        # print('Reading unique1 from file')
        assert False , 'No abstraction data' # remove this line if you want to create abstract classes
        unique1,uni1 = Abstract_classes(final_episodes,d,model)
        with open(f'{Dataset_path}/ABS/Abstraction_data_final_CI_{d}.pickle', 'wb') as f:
            pickle.dump(unique1, f)


    ######################################################### Read FRT and FRTS from pickle #############
    with open(f'{Dataset_path}/Random_episodes/FRT_test_68.pkl', 'rb') as file2:
        FRT = pickle.load(file2)
    
    ######################################################### Read model from pickle #############
    
    HD_model_path1 = f'{Dataset_path}/ML_models'
    HD_model_path = f'{HD_model_path1}/RF_FF_1rep_{d}.pickle'
    with open(HD_model_path, 'rb') as file2:
        RF_FF_1rep = pickle.load(file2)
        
    #########################################################  Plot Risk and Position #############
    if not os.path.exists("CI"):
        os.makedirs("CI")
    newpath = 'CI/Abs_{d}'
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    Num_plot = 200 #number of episodes to plot
    if len(FRT)<Num_plot:
        print("number of available episodes is less than {Num_plot}}")
        Num_plot = len(FRT)
    if d<0.05:
        Num_plot = 100
    print("Number of episodes to plot:",Num_plot)
    for i in tqdm(range(0,Num_plot,10), desc="Plotting", total=Num_plot//10):
        Forest_CI_multiple(translate_multiple_episodes_steps(FRT[i:i+10],model,translator,d,unique1),RF_FF_1rep,i,d,path=newpath)